In [ ]:
import random
import numpy as np
import csv
import re
import os 
import networkx as nx
from networkx.algorithms import approximation
from parse import read_input_file, write_output_file
from utils import is_valid_network, average_pairwise_distance
from pulp import *
import matplotlib.pyplot as plt
import time 
import random as rand

In [ ]:
G = read_input_file("25.in")

In [ ]:
len(G.nodes)

In [ ]:
def sol1(G): 
    print('entered sol 1')
    weighted_set = approximation.min_weighted_dominating_set(G)
    # need a way to connect these vertices cheapest way possible 
    all_potential_T = []
    def helper(G, T, vertices):
        nonlocal all_potential_T
        if all([a in T.nodes() for a in vertices]): 
            all_potential_T.append(T)      
        for edge in G.edges():
            if edge not in T.edges():
                if ((edge[0] not in T.nodes() and edge[1] in T.nodes()) or (edge[0] in T.nodes() and edge[1] not in T.nodes())):
                    new_T = T.copy()
                    new_T.edges()
                    new_T.add_edge(edge[0], edge[1], weight = G[edge[0]][edge[1]]["weight"])
                    if not any([nx.is_isomorphic(new_T, tree) for tree in all_potential_T]):
                        helper(G, new_T, vertices)
                        
    first_T = nx.Graph()
    first_T.add_node(weighted_set.pop())
    helper(G, first_T, weighted_set)
    return min(all_potential_T, key = lambda x: average_pairwise_distance(x))
def add_weighted_path(path, T, G): 
    for i in range(len(path)-1):
        u = path[i]
        v = path[i+1]
        if not ((u,v) in T.edges() and (v,u) in T.edges()): 
            T.add_edge(u, v, weight = G[u][v]["weight"])
    
    
# find the cheapest way to connect the weighted_set together     
def sol25(G):
    print("Started Solving")
    connectivity = {} 
    for a in G.nodes():
        neighbor_size = len(list(G.neighbors(a))) 
        if neighbor_size== len(G.nodes()) - 1: 
            T = nx.Graph() 
            T.add_node(a)
            return T 
        connectivity[a] = neighbor_size
    print('finished the iteration')
    weighted_set = approximation.min_weighted_dominating_set(G)
    weighted_set.update(set([rand.randint(0, len(G.nodes())-1) for i in range(len(G.nodes())//10)]))
    top_connect = set(sorted(connectivity.keys(), key = lambda x: connectivity[x], reverse = True)[:len(G.nodes)//10])
    weighted_set.update(top_connect) 
    weighted_set = list(weighted_set)
    all_potential_T = []
    
    
    # i think something is wrong with helper/ i was looking and im not sure how this is being called lol
    def helper(G, T, vertexes):
        nonlocal all_potential_T
        node_set = set(list(T.nodes()))
        for a in T.nodes():
            node_set.update(set(list(G.neighbors(a))))
        if len(node_set) == len(G.nodes()):
            trimmer(T) 
        else:
            for u in vertexes: 
                if u not in T.nodes():
                    for v in T.nodes():
                        new_T = T.copy()
                        try: 
                            add_weighted_path(nx.dijkstra_path(G.subgraph([v] + [a for a in G.nodes() if a not in T.nodes()]), source = u, target = v), new_T, G)
                            if nx.is_tree(new_T) and not any([nx.is_isomorphic(new_T, tree) for tree in all_potential_T]) :
                                helper(G, new_T, vertexes)
                        except:
                            a = 1
                            #print("Couldn't find appropriate path")
                            
    def trimmer(T):
        print('trim that shit')
        nonlocal all_potential_T
        for edge in T.edges():
            new_T = T.copy() 
            if len(list(T.neighbors(edge[0]))) == 1: 
                new_T.remove_edge(edge[0], edge[1])
                new_T.remove_node(edge[0])
            elif len(list(T.neighbors(edge[1]))) == 1: 
                new_T.remove_edge(edge[0], edge[1])
                new_T.remove_node(edge[1])
            node_set = set(list(new_T.nodes()))
            for a in new_T.nodes():
                node_set.update(set(list(G.neighbors(a))))
            if len(node_set) == len(G.nodes()) and average_pairwise_distance(new_T) < average_pairwise_distance(T) :
                T = new_T 
        print('boutta append T')
        all_potential_T.append(T)
        
        
    shortest_path = [nx.dijkstra_path(G, source = i, target = j ) for (i, j) in ((i, j) for i in weighted_set for j in weighted_set if i < j)]
    for path in shortest_path:
        T = nx.Graph()
        add_weighted_path(path, T, G)
        helper(G, T, weighted_set)
    return min(all_potential_T, key = lambda x: average_pairwise_distance(x))


def trimmer_back(T): 
        #nonlocal all_potential_T
        print('trim that shit back')
        for edge in T.edges():
            new_T = T.copy() 
            new_T.remove_edge(edge[0], edge[1])
            if new_T.neighbors(edge[0]) == 1: 
                new_T.remove_node(edge[0])
            else: 
                new_T.remove_node(edge[1])
            node_set = set(list(new_T.nodes()))
            for a in new_T.nodes():
                node_set.update(set(list(G.neighbors(a))))
            if len(node_set) == len(G.nodes()):
                all_potential_T.append(T) 
                trimmer(T)

def sol3(G):
    print('entered 3')
    trees = [sol25(G) for i in range(15)]
    print('finished the trees')
    return min(trees,  key = lambda x: average_pairwise_distance(x))




In [ ]:
G = read_input_file("10.in")

In [ ]:
start = time.time() 
ans = sol3(G) 
print(time.time() - start)
average_pairwise_distance(ans)

In [ ]:
pathname = '/Users/yashwagh/Desktop/Berkeley/170_proj/cs170project/inputs'
for file_name in os.scandir(pathname):
    print(file_name)

    G25 = read_input_file(file_name)
    start  = time.time()
    ans25 = sol3(G25)
    print(time.time() - start)
    G50 = read_input_file(file_name)
    start  = time.time()
    ans50 = sol3(G50)
    print(time.time() - start)
    G100 = read_input_file(file_name)
    start  = time.time()
    ans100 = sol3(G100)
    print(time.time() - start)


In [ ]:
average_pairwise_distance(ans25)

In [ ]:
def draw_graph(G): 
    pos=nx.spring_layout(G)
    plt.figure(figsize = (20, 10))
    nx.draw(G, pos)
    labels = nx.get_edge_attributes(G,'weight')
    nx.draw_networkx(G, pos,  with_labels= True)
    nx.draw_networkx_edge_labels(G,pos,edge_labels=labels, );


In [ ]:
draw_graph(G)

In [ ]:
draw_graph(ans)